In [22]:
import pymc as pm
import numpy as np
import pandas as pd
import os
import requests
import io
from datetime import datetime, timedelta

# get data
API_KEY = os.getenv("API_KEY")
url = 'https://data-service.beatthebookie.blog/data'
headers = {"x-api-key": API_KEY}

# Function to fetch data for a specific division and season
def fetch_data(division, season):
    params = {
        'division': division,
        'season': season
    }
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        return pd.read_json(io.StringIO(response.content.decode('utf-8')))
    else:
        print(f"Error fetching {division} {season}: {response.status_code}")
        print(response.content.decode('utf-8'))
        return pd.DataFrame()

# Fetch data for all combinations
seasons = ['2024_2025', '2023_2024']
divisions = ['Premier League']
dataframes = []

for division in divisions:
    for season in seasons:
        df = fetch_data(division, season)
        if not df.empty:
            dataframes.append(df)

# Combine all dataframes
if dataframes:
    df = pd.concat(dataframes, ignore_index=True)
    
    # Convert match_date to datetime
    df['match_date'] = pd.to_datetime(df['match_date'])

In [28]:
# build model

def build_bayesian_model(home_teams, away_teams, home_goals, away_goals, home_xg, away_xg, dates):
    # get unique teams
    teams = sorted(list(set(home_teams) | set(away_teams))) # alphabetically sorts and de-dupes list of team names
    team_indices = {team: idx for idx, team in enumerate(teams)} # sets index values for each team within a dict

    # convert date into time differences
    max_date = np.max(dates)
    time_diffs = (max_date - dates).dt.days

    # convert team names to index vals
    home_idx = [team_indices[team] for team in home_teams]
    away_idx = [team_indices[team] for team in away_teams]

    with pm.Model() as model:
        # initalise time decay parameter
        decay_rate = pm.HalfNormal("decay_rate", sigma=1.5/365) # balanced prior for decay rate, divided by 365 to account for daily rate

        # team strength initalisation
        attack = pm.Normal("attack", mu=0, sigma=0.3, shape=len(teams)) # initalising normal distribution for relative attacking strength with mean 0 and std of 0.3
        defense = pm.Normal('defense', mu=0, sigma=0.3, shape=len(teams))

        # initalise home advantage
        home_advantage = pm.Normal("home_advantage", mu=0.2, sigma=0.1) # initalises home_adv to 0.2 and has std of 0.1 so val can extend or reduce that much

        # create time decay factor to apply to expected goals
        time_factor = pm.math.exp(-decay_rate * time_diffs)

        # expected goals parameter for both xG and goals, applied time decay
        home_theta = time_factor * pm.math.exp(attack[home_idx] - defense[away_idx] + home_advantage) # we use exponential so it's always positive and team strengths are multiplicative
        away_theta = time_factor * pm.math.exp(attack[away_idx] - defense[home_idx])


        # goals likelihood (poisson for actual goals)
        home_goals_like = pm.Poisson("home_goals", mu=home_theta, observed=home_goals) 
        away_goals_like = pm.Poisson("away_goals", mu=away_theta, observed=away_goals)

        # xG likelihood (gamma for expected goals)
        xg_alpha = pm.HalfNormal("xg_alpha", sigma=1.0) # shape parameter (must be positive hence half normal) - alpha shapes basic form of distribution
        home_xg_beta = xg_alpha / home_theta # beta is rate parameter - scales where that form sits on the axis
        away_xg_beta = xg_alpha / away_theta # we are setting the mean of the xg distribution to be equal to our team strength rating

        home_xg_like = pm.Gamma("home_xg", alpha=xg_alpha, beta=home_xg_beta, observed=home_xg)
        away_xg_like = pm.Gamma("away_xg", alpha=xg_alpha, beta=away_xg_beta, observed=away_xg)


    return model, team_indices

def fit_bayesian_model(model, draws=1000):
    with model:
        trace = pm.sample(draws=draws) # x draws means the MCMC is generating x different sets of parameters that are ALL plausible given our data, with more likely parameter sets appearing more frequently in our draws.
    return trace


def get_team_strengths(trace, team_indices, current_teams):
    teams = list(team_indices.keys())
    attack_means = trace.posterior['attack'].mean(dim=['chain', 'draw']).values
    defense_means = trace.posterior['defense'].mean(dim=['chain', 'draw']).values
    home_adv = trace.posterior['home_advantage'].mean(dim=['chain', 'draw']).values
    
    # Create DataFrame with better formatting
    results = pd.DataFrame({
        'team': teams,
        'attack_strength': attack_means,
        'defense_strength': defense_means,
        'overall_strength': (np.exp(attack_means - np.mean(defense_means)) - 
                    np.exp(np.mean(attack_means) - defense_means)), #this calculates the expected goal difference of team vs average league team in a match with no home adv
        'home_advantage': home_adv

    })

    # filter out relegated teams
    results = results[results['team'].isin(current_teams)]
    
    # Round the values and sort by overall strength
    results = results.round(3).sort_values('overall_strength', ascending=False)
    
    return results, home_adv
    

In [27]:
data = df[["home_team", "away_team", "home_goals", "away_goals", "home_xgoals", "away_xgoals", "match_date"]]

# filter to matches only in previous 365 days
data = data[data["match_date"] > datetime.now() - timedelta(days=365)]

# get list of current teams
current_teams = df[df["season"] == 20242025]["home_team"].unique()

# Build model
model, team_indices = build_bayesian_model(
        home_teams=data['home_team'],
        away_teams=data['away_team'],
        home_goals=np.array(data['home_goals']),
        away_goals=np.array(data['away_goals']),
        home_xg=np.array(data["home_xgoals"]),
        away_xg=np.array(data["away_xgoals"]),
        dates=data["match_date"]
    )
    
# Fit model
trace = fit_bayesian_model(model)
    
# Get results
strengths, home_adv = get_team_strengths(trace, team_indices, current_teams)
print("\nTeam Strengths:")
print(strengths.to_string(index=False))

print(f"Home advantage: {home_adv}")

c:\Users\Owner\dev\football-analytics\.venv\Lib\site-packages\pytensor\tensor\rewriting\elemwise.py:1027: UserWarning: Loop fusion failed because the resulting node would exceed the kernel argument limit.
  warn(
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
c:\Users\Owner\dev\football-analytics\.venv\Lib\site-packages\pytensor\tensor\rewriting\elemwise.py:1027: UserWarning: Loop fusion failed because the resulting node would exceed the kernel argument limit.
  warn(
c:\Users\Owner\dev\football-analytics\.venv\Lib\site-packages\pytensor\tensor\rewriting\elemwise.py:1027: UserWarning: Loop fusion failed because the resulting node would exceed the kernel argument limit.
  warn(
c:\Users\Owner\dev\football-analytics\.venv\Lib\site-packages\pytensor\tensor\rewriting\elemwise.py:1027: UserWarning: Loop fusion failed because the resulting node would exceed the kernel argument limit.
  warn(
c:\Users\Owner\dev\football-analytics\.venv\Lib\site-packages\pytensor\t

c:\Users\Owner\dev\football-analytics\.venv\Lib\site-packages\rich\live.py:231: UserWarning: install "ipywidgets" 
for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 254 seconds.
There were 1 divergences after tuning. Increase `target_accept` or reparameterize.


ValueError: too many values to unpack (expected 2)

In [29]:
# Get results
strengths, home_adv = get_team_strengths(trace, team_indices, current_teams)
print("\nTeam Strengths:")
print(strengths.to_string(index=False))

print(f"Home advantage: {home_adv}")


Team Strengths:
          team  attack_strength  defense_strength  overall_strength  home_advantage
     Liverpool            0.638             0.162             1.243           0.181
       Arsenal            0.502             0.472             1.226           0.181
      Man City            0.538             0.110             0.971           0.181
       Chelsea            0.530            -0.122             0.676           0.181
     Newcastle            0.400            -0.072             0.495           0.181
   Bournemouth            0.392            -0.085             0.464           0.181
     Tottenham            0.409            -0.206             0.323           0.181
 Nott'm Forest            0.130             0.027             0.192           0.181
        Fulham            0.176            -0.032             0.187           0.181
Crystal Palace            0.176            -0.052             0.160           0.181
      Brighton            0.146            -0.037          